In [ ]:
import torch_directml
from tokenizers import Tokenizer
from tokenizers.normalizers import Sequence, Strip
from datasets import load_dataset
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import ByteLevel

In [2]:
device = torch_directml.device()
# device = torch.device("cpu")
print(device)

privateuseone:0


In [3]:
dataset = load_dataset("OleehyO/latex-formulas", "cleaned_formulas")["train"]  # type: ignore

In [4]:
formulas = [x["latex_formula"] for x in dataset]  # type: ignore
print(formulas[0])

\begin{align*}-\div (y^{1-2m}\nabla w)=0\mbox{in}\mathbb R^n\times\mathbb R_+;w\big|_{y=0}=|u|\end{align*}


In [7]:
tokenizer = Tokenizer(BPE(unk_token="<unk>"))

trainer = BpeTrainer(
    vocab_size=8000,  # type: ignore
    special_tokens=["<pad>", "<sos>", "<eos>", "<unk>"],  # type: ignore
)

tokenizer.pre_tokenizer = ByteLevel()  # type: ignore
tokenizer.normalizer = Sequence([Strip()])  # type: ignore
# tokenizer.post_processor = TemplateProcessing(  # type: ignore
#     single="<sos> $A <eos>",
#     pair="<sos> $A <eos> <sos> $B <eos>",
#     special_tokens=[
#         ("<sos>", tokenizer.token_to_id("<sos>")),
#         ("<eos>", tokenizer.token_to_id("<eos>")),
#     ],
# )

In [8]:
tokenizer.train_from_iterator(formulas, trainer=trainer)
tokenizer.save("latex-tokenizer.json")